# LargeST dataset for sequence
## Note:
### The dataset concludes 2019, 2020, 2021 traffic flow data of freeways in LA.
### We currently set the data for the specific day 12.24, time slot in 8:00-11:00 a.m. as input data, which has 12 flows in length. The next one hour data is considered as labels in prediction tasks.

###You could train the sequence prediction task using the preocessed data. Input data and labels for three years are listed in CSV files.

###These settings coulf be changed as your requirements. Please check the code and origin data.

### Author: Qiming Zhang

In [1]:
# import os
# import re
# import torch
# import argparse
# import bitsandbytes as bnb
# from trl import SFTTrainer
# from functools import partial
# from datasets import load_dataset

In [ ]:
# !huggingface-cli login

In [ ]:
# !huggingface-cli whoami

In [2]:
# from datasets import Dataset
import numpy as np
import pandas as pd

ca_meta = pd.read_csv('./Desktop/LargeST/data/ca/ca_meta.csv')
gla_meta = ca_meta[(ca_meta.District == 7) | (ca_meta.District == 8) | (ca_meta.District == 12)]
gla_meta = gla_meta.reset_index()
gla_meta = gla_meta.drop(columns=['index'])
gla_meta.to_csv('gla_meta.csv', index=False)
print(gla_meta[gla_meta.duplicated(subset=['Lat', 'Lng'])])
gla_meta

Empty DataFrame
Columns: [ID, Lat, Lng, District, County, Fwy, Lanes, Type, Direction, ID2]
Index: []


,ID,Lat,Lng,District,County,Fwy,Lanes,Type,Direction,ID2
0,767494,34.103332,-118.249733,7,Los Angeles,SR2-E,3,Mainline,E,3527
1,767509,34.107693,-118.249538,7,Los Angeles,SR2-E,4,Mainline,E,3528
2,767523,34.113959,-118.242956,7,Los Angeles,SR2-E,4,Mainline,E,3529
3,767541,34.116060,-118.238384,7,Los Angeles,SR2-E,4,Mainline,E,3530
4,767554,34.121074,-118.229892,7,Los Angeles,SR2-E,4,Mainline,E,3531
...,...,...,...,...,...,...,...,...,...,...
3829,1202527,33.800970,-118.082822,12,Orange,I605-N,4,Mainline,N,8595
3830,1202549,33.804853,-118.081293,12,Orange,I605-N,4,Mainline,N,8596
3831,1219560,33.799259,-118.084891,12,Orange,I605-S,5,Mainline,S,8597
3832,1202522,33.801040,-118.083136,12,Orange,I605-S,4,Mainline,S,8598


In [56]:
new_df = gla_meta.set_index('ID')[['Lat', 'Lng']].transpose().reindex()
new_df

ID,767494,767509,767523,767541,767554,767572,767620,767455,768202,767470,...,1221523,1221550,1221536,1221556,1219551,1202527,1202549,1219560,1202522,1202537
Lat,34.103332,34.107693,34.113959,34.116060,34.121074,34.128043,34.135337,34.146613,34.147961,34.155998,...,33.774330,33.774743,33.778783,33.782664,33.799174,33.800970,33.804853,33.799259,33.801040,33.804047
Lng,-118.249733,-118.249538,-118.242956,-118.238384,-118.229892,-118.228421,-118.229285,-118.226176,-118.225767,-118.224185,...,-118.083008,-118.085896,-118.090036,-118.091096,-118.084562,-118.082822,-118.081293,-118.084891,-118.083136,-118.081675


In [4]:
year = '2021'  # please specify the year, our experiments use 2019

gla_meta.ID = gla_meta.ID.astype(str)
gla_meta_id = gla_meta.ID.values.tolist()

ca_his = pd.read_hdf('./Desktop/LargeST/data/ca/ca_his_' + year +'.h5')
gla_his = ca_his[gla_meta_id]
# gla_his
gla_his.to_hdf('gla_his_' + year + '.h5', key='t', mode='w')
gla_his.shape

(35040, 3834)

In [ ]:
# gla_his._append(new_df)

In [5]:
year = '2020'  # please specify the year, our experiments use 2019

gla_meta.ID = gla_meta.ID.astype(str)
gla_meta_id = gla_meta.ID.values.tolist()

ca_his_20 = pd.read_hdf('./Desktop/LargeST/data/ca/ca_his_' + year +'.h5')
gla_his_20 = ca_his_20[gla_meta_id]
gla_his_20
gla_his_20.to_hdf('gla_his_' + year + '.h5', key='t', mode='w')
gla_his_20.shape

(35136, 3834)

In [6]:
year = '2019'  # please specify the year, our experiments use 2019

gla_meta.ID = gla_meta.ID.astype(str)
gla_meta_id = gla_meta.ID.values.tolist()

ca_his_19 = pd.read_hdf('./Desktop/LargeST/data/ca/ca_his_' + year +'.h5')
gla_his_19 = ca_his_19[gla_meta_id]
gla_his_19
gla_his_19.to_hdf('gla_his_' + year + '.h5', key='t', mode='w')
gla_his_19.shape

(35040, 3834)

In [7]:
data_all = pd.concat([gla_his_19, gla_his_20, gla_his], axis=0, ignore_index=False)
# data_all.to_csv("gla_his_19-21.csv")
data_all.shape

(105216, 3834)

In [8]:
data_all.index

DatetimeIndex(['2019-01-01 00:00:00', '2019-01-01 00:15:00',
               '2019-01-01 00:30:00', '2019-01-01 00:45:00',
               '2019-01-01 01:00:00', '2019-01-01 01:15:00',
               '2019-01-01 01:30:00', '2019-01-01 01:45:00',
               '2019-01-01 02:00:00', '2019-01-01 02:15:00',
               ...
               '2021-12-31 21:30:00', '2021-12-31 21:45:00',
               '2021-12-31 22:00:00', '2021-12-31 22:15:00',
               '2021-12-31 22:30:00', '2021-12-31 22:45:00',
               '2021-12-31 23:00:00', '2021-12-31 23:15:00',
               '2021-12-31 23:30:00', '2021-12-31 23:45:00'],
              dtype='datetime64[ns]', name='Time', length=105216, freq='15T')

In [27]:
# select specific date data: if you want to change date, please change the specific para
def select_data_time(df, start_time='08:00:00', end_time='12:00:00', month=12, day=24):

    """
    start_time, end_time: type: string
    month, day: type: int

    """
    start_time = pd.to_datetime(start_time).time()
    end_time = pd.to_datetime(end_time).time()
    date_data = df[(df.index.year.isin([2019, 2020, 2021])) &
                        (df.index.month == month) &
                        (df.index.day == day) &
                        (df.index.time >= start_time) &
                        (df.index.time < end_time)]

    return date_data


In [28]:
print(select_data_time(data_all))

                     767494  767509  767523  767541  767554  767572  767620  \
Time                                                                          
2019-12-24 08:00:00    77.0   199.0   191.0   223.0   191.0   213.0   175.0   
2019-12-24 08:15:00    75.0   196.0   189.0   232.0   201.0   224.0   183.0   
2019-12-24 08:30:00    84.0   192.0   186.0   224.0   188.0   205.0   175.0   
2019-12-24 08:45:00    73.0   222.0   216.0   251.0   217.0   241.0   198.0   
2019-12-24 09:00:00    75.0   194.0   186.0   223.0   181.0   211.0   170.0   
2019-12-24 09:15:00    92.0   221.0   209.0   260.0   232.0   238.0   196.0   
2019-12-24 09:30:00    83.0   234.0   217.0   266.0   223.0   246.0   208.0   
2019-12-24 09:45:00   101.0   244.0   229.0   277.0   234.0   259.0   206.0   
2019-12-24 10:00:00    89.0   230.0   223.0   281.0   234.0   266.0   213.0   
2019-12-24 10:15:00    99.0   255.0   256.0   302.0   258.0   284.0   233.0   
2019-12-24 10:30:00    93.0   250.0   250.0   309.0 

In [59]:
# Formatted output

def format_column_instructions(column):
    formatted_values = [str(int(value)) for value in column]
    str_re_2019 = ', '.join(formatted_values[:16])
    str_re_2020 = ', '.join(formatted_values[16:32])
    str_re_2021 = ', '.join(formatted_values[32:-4]) # slice index should be changed if you wanna select defferent time data

    output = f'\n2019-10-31: {str_re_2019}\n2020-10-31: {str_re_2020}\n'

    prompt_descriptions = 'You are a traffic sensor node, which can record the traffic volume for each 15 minutes. \
    The history volume data of your from 8:00 a.m. to 10:45 a.m. on 2021.10.31 is provided: '+str_re_2021 +'. \
    We have past two year data of you from 8:00 a.m. to 12:00 a.m. on 10.31 for reference: '+output+' \
    Please predict the traffic volume for each 15-minute in next one hour on 2021.10.31:'

    return prompt_descriptions

def format_column_answer(column):
    # formatted_values = str(int(column.iloc[-1]))
    formatted_values = [str(int(value)) for value in column.iloc[-4:]]
    str_re = ', '.join(formatted_values)
    prompt_descriptions = "The predicted volumes for next one hour are "+str_re+"."
    return prompt_descriptions

def format_column_text(column):
    ins = format_column_instructions(column)
    ans = format_column_answer(column)
    text = "<s>[INST] "+ins+" [/INST] "+ans+" </s>"
    return text

def format_column_text_num(column):
    ins = format_column_instructions_num(column)
    ans = format_column_answer_num(column)
    text_num = "<s>[INST] "+ins+" [/INST] "+ans+" </s>"
    return text_num



# input data and labels: note: slice should be changed if time slots are changed
def format_column_instructions_num(column):

    formatted_values = [str(int(value)) for value in column]
    str_re_2019 = ', '.join(formatted_values[:12])
    str_re_2020 = ', '.join(formatted_values[16:28])
    str_re_2021 = ', '.join(formatted_values[32:-4])

    str_re_2019_la = ', '.join(formatted_values[12:16])
    str_re_2020_la = ', '.join(formatted_values[28:32])
    str_re_2021_la = ', '.join(formatted_values[-4:])

    return (str_re_2019, str_re_2020, str_re_2021, str_re_2019_la, str_re_2020_la, str_re_2021_la)


def generate_sft_dataset(df, new_df, para = 0.8):

    # input : length 12  time slot: 8-11
    # output: leng 4 time slot: 11-12
    # new_df: additional data: here are locations
    data_set = select_data_time(df)
    data = data_set.apply(format_column_instructions_num)
    print(type(data))

    new_index_mapping = {0: 'data_2019', 1: 'data_2020', 2: 'data_2021', 3: 'label_2019', 4: 'label_2020', 5: 'label_2021'}

    # rename index
    df = data.rename(index=new_index_mapping)
    df = df._append(new_df)
    print(df)

    data_set.shape[1] # 3834
    id = int(data_set.shape[1]*para)
    print(id)

    df.to_csv("data.csv")
    train = df.iloc[:,:id]
    test = df.iloc[:,id:]

    print(train)
    print(test)

    train.to_csv("train.csv")
    test.to_csv("test.csv")

    print("Files Saved!")

In [60]:
generate_sft_dataset(data_all, new_df, 0.8)

<class 'pandas.core.frame.DataFrame'>
                                                       767494  \
data_2019     77, 75, 84, 73, 75, 92, 83, 101, 89, 99, 93, 93   
data_2020   89, 83, 101, 101, 83, 101, 50, 50, 63, 102, 13...   
data_2021   63, 77, 104, 70, 115, 95, 112, 127, 126, 141, ...   
label_2019                                 114, 136, 110, 115   
label_2020                                 141, 125, 119, 119   
label_2021                                 164, 171, 148, 161   
Lat                                                 34.103332   
Lng                                               -118.249733   

                                                       767509  \
data_2019   199, 196, 192, 222, 194, 221, 234, 244, 230, 2...   
data_2020   187, 187, 189, 204, 199, 208, 230, 243, 268, 2...   
data_2021   131, 137, 158, 157, 173, 164, 175, 196, 185, 2...   
label_2019                                 249, 293, 293, 320   
label_2020                                 289, 271

In [61]:
df = pd.read_csv('data.csv')
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df

,Unnamed: 0,767494,767509,767523,767541,767554,767572,767620,767455,768202,...,1221523,1221550,1221536,1221556,1219551,1202527,1202549,1219560,1202522,1202537
0,data_2019,"77, 75, 84, 73, 75, 92, 83, 101, 89, 99, 93, 93","199, 196, 192, 222, 194, 221, 234, 244, 230, 2...","191, 189, 186, 216, 186, 209, 217, 229, 223, 2...","223, 232, 224, 251, 223, 260, 266, 277, 281, 3...","191, 201, 188, 217, 181, 232, 223, 234, 234, 2...","213, 224, 205, 241, 211, 238, 246, 259, 266, 2...","175, 183, 175, 198, 170, 196, 208, 206, 213, 2...","85, 89, 81, 90, 85, 93, 100, 90, 105, 108, 105...","75, 84, 74, 81, 84, 90, 91, 85, 98, 95, 100, 107",...,"427, 467, 471, 439, 417, 478, 521, 498, 481, 5...","443, 482, 477, 466, 439, 482, 533, 540, 522, 5...","276, 304, 309, 301, 288, 321, 345, 360, 366, 3...","289, 306, 321, 305, 294, 335, 350, 370, 368, 3...","303, 308, 340, 348, 337, 366, 355, 396, 368, 3...","252, 258, 289, 286, 294, 306, 309, 346, 296, 3...","254, 253, 283, 275, 284, 311, 309, 328, 311, 3...","276, 302, 282, 300, 281, 309, 318, 293, 228, 2...","232, 252, 241, 253, 241, 264, 269, 253, 241, 2...","257, 276, 261, 266, 256, 307, 280, 267, 266, 2..."
1,data_2020,"89, 83, 101, 101, 83, 101, 50, 50, 63, 102, 13...","187, 187, 189, 204, 199, 208, 230, 243, 268, 2...","186, 161, 187, 199, 192, 193, 216, 224, 235, 2...","258, 211, 265, 261, 254, 275, 315, 319, 322, 3...","123, 123, 122, 121, 117, 117, 115, 114, 110, 1...","230, 210, 216, 237, 233, 216, 217, 253, 270, 2...","150, 142, 143, 173, 156, 151, 167, 184, 205, 1...","103, 93, 105, 109, 101, 103, 99, 126, 132, 123...","75, 68, 75, 84, 72, 70, 78, 97, 87, 88, 101, 101",...,"439, 441, 482, 484, 480, 496, 539, 525, 565, 5...","367, 360, 409, 377, 386, 406, 439, 435, 470, 4...","239, 235, 270, 244, 256, 272, 298, 280, 305, 3...","247, 233, 274, 261, 256, 283, 307, 274, 306, 3...","222, 244, 267, 286, 245, 273, 314, 312, 320, 3...","185, 193, 223, 248, 196, 226, 245, 291, 290, 2...","178, 196, 209, 211, 205, 223, 253, 252, 276, 2...","313, 335, 348, 378, 323, 366, 438, 434, 409, 4...","202, 240, 255, 252, 225, 273, 296, 285, 274, 2...","187, 222, 206, 236, 204, 230, 258, 274, 268, 2..."
2,data_2021,"63, 77, 104, 70, 115, 95, 112, 127, 126, 141, ...","131, 137, 158, 157, 173, 164, 175, 196, 185, 2...","142, 141, 156, 164, 164, 171, 170, 201, 185, 2...","178, 164, 170, 187, 200, 194, 210, 244, 221, 2...","211, 174, 178, 199, 219, 209, 228, 259, 249, 2...","145, 155, 163, 165, 204, 190, 196, 242, 215, 2...","142, 147, 139, 181, 221, 182, 168, 181, 181, 1...","46, 61, 53, 66, 94, 73, 67, 80, 74, 83, 81, 98","46, 59, 46, 58, 70, 63, 60, 77, 71, 74, 75, 90",...,"441, 475, 485, 521, 522, 551, 603, 633, 616, 6...","306, 320, 343, 382, 370, 389, 438, 469, 448, 4...","222, 224, 233, 251, 242, 255, 306, 324, 305, 3...","224, 228, 237, 246, 227, 271, 313, 332, 312, 3...","194, 199, 230, 275, 258, 276, 317, 322, 316, 3...","167, 174, 194, 228, 219, 244, 272, 269, 265, 2...","164, 210, 194, 225, 213, 239, 265, 262, 258, 2...","241, 303, 308, 345, 338, 367, 395, 426, 408, 4...","162, 227, 220, 215, 218, 224, 227, 244, 257, 2...","153, 235, 193, 205, 208, 225, 226, 250, 242, 2..."
3,label_2019,"114, 136, 110, 115","249, 293, 293, 320","257, 301, 307, 323","313, 355, 367, 389","264, 298, 305, 320","280, 339, 330, 362","231, 276, 289, 286","117, 135, 136, 146","107, 124, 126, 140",...,"568, 560, 617, 617","622, 617, 665, 658","429, 438, 481, 465","443, 449, 492, 487","412, 432, 421, 432","359, 368, 362, 367","345, 357, 358, 351","325, 332, 334, 362","271, 286, 299, 299","277, 305, 301, 296"
4,label_2020,"141, 125, 119, 119","289, 271, 272, 299","260, 244, 273, 305","341, 325, 357, 393","109, 112, 112, 111","277, 246, 277, 325","210, 198, 220, 245","130, 126, 144, 159","99, 95, 117, 128",...,"655, 682, 740, 722","541, 566, 620, 609","369, 383, 425, 416","387, 413, 437, 442","365, 396, 400, 389","304, 343, 347, 329","296, 328, 311, 320","461, 462, 503, 494","314, 304, 345, 341","293, 293, 255, 316"
5,label_2

In [62]:
df_train

,Unnamed: 0,767494,767509,767523,767541,767554,767572,767620,767455,768202,...,1205200,1205211,1205230,1205246,1205276,1205286,1205297,1205310,1205324,1205335
0,data_2019,"77, 75, 84, 73, 75, 92, 83, 101, 89, 99, 93, 93","199, 196, 192, 222, 194, 221, 234, 244, 230, 2...","191, 189, 186, 216, 186, 209, 217, 229, 223, 2...","223, 232, 224, 251, 223, 260, 266, 277, 281, 3...","191, 201, 188, 217, 181, 232, 223, 234, 234, 2...","213, 224, 205, 241, 211, 238, 246, 259, 266, 2...","175, 183, 175, 198, 170, 196, 208, 206, 213, 2...","85, 89, 81, 90, 85, 93, 100, 90, 105, 108, 105...","75, 84, 74, 81, 84, 90, 91, 85, 98, 95, 100, 107",...,"606, 612, 662, 648, 582, 635, 639, 623, 596, 6...","612, 612, 666, 652, 582, 630, 646, 621, 601, 6...","546, 544, 592, 571, 528, 565, 579, 549, 556, 6...","627, 635, 696, 672, 630, 647, 668, 631, 633, 6...","594, 599, 656, 646, 596, 608, 641, 609, 598, 6...","382, 412, 452, 439, 397, 422, 459, 440, 423, 4...","410, 429, 455, 449, 417, 444, 448, 433, 416, 4...","317, 333, 365, 335, 317, 345, 350, 331, 332, 3...","320, 338, 361, 341, 318, 351, 360, 335, 340, 3...","354, 371, 389, 370, 347, 378, 380, 367, 372, 3..."
1,data_2020,"89, 83, 101, 101, 83, 101, 50, 50, 63, 102, 13...","187, 187, 189, 204, 199, 208, 230, 243, 268, 2...","186, 161, 187, 199, 192, 193, 216, 224, 235, 2...","258, 211, 265, 261, 254, 275, 315, 319, 322, 3...","123, 123, 122, 121, 117, 117, 115, 114, 110, 1...","230, 210, 216, 237, 233, 216, 217, 253, 270, 2...","150, 142, 143, 173, 156, 151, 167, 184, 205, 1...","103, 93, 105, 109, 101, 103, 99, 126, 132, 123...","75, 68, 75, 84, 72, 70, 78, 97, 87, 88, 101, 101",...,"501, 513, 564, 557, 536, 541, 588, 604, 561, 6...","472, 512, 564, 559, 522, 520, 593, 595, 540, 6...","410, 453, 538, 554, 522, 498, 523, 579, 512, 6...","407, 450, 532, 539, 484, 488, 522, 558, 495, 5...","439, 486, 518, 525, 481, 478, 530, 542, 491, 5...","337, 372, 407, 401, 372, 384, 416, 418, 399, 4...","340, 375, 397, 399, 372, 371, 408, 404, 382, 4...","271, 276, 285, 308, 266, 273, 320, 296, 278, 3...","268, 269, 315, 312, 276, 282, 332, 309, 285, 3...","290, 290, 330, 324, 304, 311, 353, 342, 314, 3..."
2,data_2021,"63, 77, 104, 70, 115, 95, 112, 127, 126, 141, ...","131, 137, 158, 157, 173, 164, 175, 196, 185, 2...","142, 141, 156, 164, 164, 171, 170, 201, 185, 2...","178, 164, 170, 187, 200, 194, 210, 244, 221, 2...","211, 174, 178, 199, 219, 209, 228, 259, 249, 2...","145, 155, 163, 165, 204, 190, 196, 242, 215, 2...","142, 147, 139, 181, 221, 182, 168, 181, 181, 1...","46, 61, 53, 66, 94, 73, 67, 80, 74, 83, 81, 98","46, 59, 46, 58, 70, 63, 60, 77, 71, 74, 75, 90",...,"386, 440, 487, 491, 476, 543, 593, 597, 565, 6...","386, 437, 490, 485, 471, 543, 592, 598, 565, 6...","374, 425, 478, 483, 460, 533, 587, 580, 554, 6...","385, 453, 593, 487, 464, 543, 571, 590, 559, 6...","346, 395, 434, 435, 429, 500, 531, 535, 513, 5...","257, 302, 325, 318, 322, 366, 381, 386, 376, 4...","279, 337, 371, 349, 355, 411, 429, 422, 425, 4...","206, 254, 261, 252, 266, 304, 320, 311, 321, 3...","210, 261, 266, 263, 273, 312, 324, 319, 327, 3...","225, 273, 284, 281, 284, 332, 350, 342, 351, 3..."
3,label_2019,"114, 136, 110, 115","249, 293, 293, 320","257, 301, 307, 323","313, 355, 367, 389","264, 298, 305, 320","280, 339, 330, 362","231, 276, 289, 286","117, 135, 136, 146","107, 124, 126, 140",...,"685, 690, 695, 694","682, 692, 695, 697","610, 600, 614, 602","707, 692, 730, 698","668, 652, 681, 642","473, 477, 468, 435","468, 495, 485, 465","361, 395, 368, 379","378, 397, 388, 388","398, 433, 399, 415"
4,label_2020,"141, 125, 119, 119","289, 271, 272, 299","260, 244, 273, 305","341, 325, 357, 393","109, 112, 112, 111","277, 246, 277, 325","210, 198, 220, 245","130, 126, 144, 159","99, 95, 117, 128",...,"637, 691, 676, 686","618, 686, 662, 672","609, 654, 657, 630","574, 638, 604, 613","574, 617, 584, 651","472, 516, 484, 549","437, 486, 450, 493","328, 370, 339, 369","340, 386, 348, 372","372, 417, 386, 400"
5,label_2

In [63]:
df_test

,Unnamed: 0,1205349,1205364,1212094,1205384,1205395,1205421,1212116,1212218,1205440,...,1221523,1221550,1221536,1221556,1219551,1202527,1202549,1219560,1202522,1202537
0,data_2019,"447, 465, 484, 470, 433, 482, 483, 515, 517, 5...","473, 502, 522, 477, 441, 489, 484, 519, 536, 5...","427, 437, 474, 428, 417, 456, 448, 482, 475, 4...","394, 393, 424, 407, 390, 427, 428, 459, 462, 4...","442, 446, 469, 440, 429, 463, 462, 483, 474, 4...","427, 435, 445, 429, 408, 436, 454, 466, 450, 4...","455, 452, 462, 453, 426, 452, 470, 500, 453, 4...","491, 498, 506, 491, 459, 500, 500, 533, 506, 5...","367, 374, 377, 365, 331, 350, 331, 363, 323, 3...",...,"427, 467, 471, 439, 417, 478, 521, 498, 481, 5...","443, 482, 477, 466, 439, 482, 533, 540, 522, 5...","276, 304, 309, 301, 288, 321, 345, 360, 366, 3...","289, 306, 321, 305, 294, 335, 350, 370, 368, 3...","303, 308, 340, 348, 337, 366, 355, 396, 368, 3...","252, 258, 289, 286, 294, 306, 309, 346, 296, 3...","254, 253, 283, 275, 284, 311, 309, 328, 311, 3...","276, 302, 282, 300, 281, 309, 318, 293, 228, 2...","232, 252, 241, 253, 241, 264, 269, 253, 241, 2...","257, 276, 261, 266, 256, 307, 280, 267, 266, 2..."
1,data_2020,"377, 376, 414, 444, 403, 416, 457, 430, 425, 4...","402, 379, 460, 444, 390, 426, 464, 448, 412, 4...","357, 356, 401, 405, 362, 389, 433, 405, 387, 4...","310, 317, 336, 383, 332, 355, 395, 362, 352, 3...","383, 395, 407, 440, 397, 390, 454, 412, 421, 4...","348, 343, 373, 390, 353, 347, 407, 380, 396, 4...","368, 367, 407, 420, 390, 388, 458, 413, 415, 4...","407, 395, 445, 459, 432, 417, 508, 458, 447, 4...","366, 364, 392, 420, 384, 372, 452, 417, 401, 4...",...,"439, 441, 482, 484, 480, 496, 539, 525, 565, 5...","367, 360, 409, 377, 386, 406, 439, 435, 470, 4...","239, 235, 270, 244, 256, 272, 298, 280, 305, 3...","247, 233, 274, 261, 256, 283, 307, 274, 306, 3...","222, 244, 267, 286, 245, 273, 314, 312, 320, 3...","185, 193, 223, 248, 196, 226, 245, 291, 290, 2...","178, 196, 209, 211, 205, 223, 253, 252, 276, 2...","313, 335, 348, 378, 323, 366, 438, 434, 409, 4...","202, 240, 255, 252, 225, 273, 296, 285, 274, 2...","187, 222, 206, 236, 204, 230, 258, 274, 268, 2..."
2,data_2021,"252, 279, 299, 293, 290, 325, 349, 339, 350, 3...","520, 525, 522, 511, 490, 501, 503, 497, 484, 4...","282, 304, 340, 359, 358, 380, 388, 410, 410, 4...","405, 451, 485, 492, 491, 554, 561, 573, 560, 5...","253, 285, 316, 329, 328, 357, 366, 375, 376, 3...","404, 407, 444, 456, 413, 452, 477, 464, 479, 4...","322, 328, 399, 432, 402, 453, 472, 525, 513, 5...","342, 379, 430, 450, 438, 503, 517, 556, 530, 5...","277, 312, 353, 361, 362, 412, 418, 460, 429, 4...",...,"441, 475, 485, 521, 522, 551, 603, 633, 616, 6...","306, 320, 343, 382, 370, 389, 438, 469, 448, 4...","222, 224, 233, 251, 242, 255, 306, 324, 305, 3...","224, 228, 237, 246, 227, 271, 313, 332, 312, 3...","194, 199, 230, 275, 258, 276, 317, 322, 316, 3...","167, 174, 194, 228, 219, 244, 272, 269, 265, 2...","164, 210, 194, 225, 213, 239, 265, 262, 258, 2...","241, 303, 308, 345, 338, 367, 395, 426, 408, 4...","162, 227, 220, 215, 218, 224, 227, 244, 257, 2...","153, 235, 193, 205, 208, 225, 226, 250, 242, 2..."
3,label_2019,"521, 555, 552, 534","544, 567, 536, 536","496, 516, 500, 495","484, 489, 486, 486","500, 516, 511, 492","472, 494, 488, 466","489, 539, 501, 512","536, 553, 531, 540","401, 417, 413, 434",...,"568, 560, 617, 617","622, 617, 665, 658","429, 438, 481, 465","443, 449, 492, 487","412, 432, 421, 432","359, 368, 362, 367","345, 357, 358, 351","325, 332, 334, 362","271, 286, 299, 299","277, 305, 301, 296"
4,label_2020,"482, 531, 509, 539","475, 534, 511, 519","441, 479, 470, 484","432, 449, 462, 459","495, 524, 534, 543","443, 451, 453, 482","469, 482, 489, 519","510, 525, 535, 568","474, 472, 485, 502",...,"655, 682, 740, 722","541, 566, 620, 609","369, 383, 425, 416","387, 413, 437, 442","365, 396, 400, 389","304, 343, 347, 329","296, 328, 311, 320","461, 462, 503, 494","314, 304, 345, 341","293, 293